In [2]:
import tkinter as tk

In [3]:
class Estado:
    def __init__(self, fila, columna, costo, padre):
        self.fila = fila
        self.columna = columna
        self.costo = costo
        self.padre = padre

    def __lt__(self, other):
        # Comparación basada en el costo total (costo acumulado + costo estimado)
        return (self.costo) < (other.costo)

In [4]:
board_size = 8
white_color = 'white'
black_color = 'black'
default_color = 'lightblue'
board = [[' ' for _ in range(board_size)] for _ in range(board_size)]
board[3][3] = board[4][4] = 'W'
board[3][4] = board[4][3] = 'B'
print(board)

[[' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', 'W', 'B', ' ', ' ', ' '], [' ', ' ', ' ', 'B', 'W', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' '], [' ', ' ', ' ', ' ', ' ', ' ', ' ', ' ']]


In [5]:
points_direction = {
    0: 'N',
    1: 'NE',
    2: 'E',
    3: 'SE',
    4: 'S',
    5: 'SO',
    6: 'O',
    7: 'NO'
}

directions = [(-1, 0), (-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1)]

def check_mov_direction(row, col, player):
    pos = 0
    direction_check = []
    check = False
    if player == "black":
        openent = 'W'
    else:
        openent = 'B'
    for dr, dc in directions:
        if 0 <= row + dr < board_size and 0 <= col + dc < board_size:  # Verificar los límites
            if board[row + dr][col + dc] == openent:
                direction_check.append(points_direction[pos])
                check = True
        pos += 1
    print("Direcciones disponibles", direction_check)
    return direction_check, check


def verify_limiter(x,y):
    return (x>-1 or y>-1 or x<8 or y <8)

def check_mov(row, col, dir, player):
    coor_repaint = []
    coor_repaint.append((row, col))
    if player == "black":
        openent = 'W'
        player = 'B'
    else:
        player = 'W'
        openent = 'B'
    y, x = row, col
    print("La dirección", dir, "filas y col", row, col)
    while True:  # Utilizar un bucle para verificar las condiciones de límite en cada paso
        if dir == 'N':
            y -= 1
        elif dir == 'NE':
            y -= 1
            x += 1
        elif dir == 'E':
            x += 1
        elif dir == 'SE':
            y += 1
            x += 1
        elif dir == 'S':
            y += 1
        elif dir == 'SO':
            y += 1
            x -= 1
        elif dir == 'O':
            x -= 1
        elif dir == 'NO':
            y -= 1
            x -= 1
        else:
            break
        
        if 0 <= x < board_size and 0 <= y < board_size:
            if board[y][x] == openent:
                coor_repaint.append((y, x))
            else:
                if 0 <= x < board_size and 0 <= y < board_size:
                    if board[y][x] == player:
                        break
                else:
                    break
        else:
            break

    print("Las coordenadas a repintar son", coor_repaint)
    return coor_repaint

def coor_to_paint(row, col, direction_check, player):
    result = []
    for dir in direction_check:
        res = check_mov(row, col, dir, player)
        if res != []:
            result.append(res)
            print("Coordenadas para pintar", result)
    return result


In [6]:
def finalizar_juego():
    if not hay_posiciones_libres() or (not hay_jugadas_disponibles('W') and not hay_jugadas_disponibles('B')):
        contar_fichas()
        root.quit()  # Finaliza la aplicación

def hay_posiciones_libres():
    for row in range(board_size):
        for col in range(board_size):
            if board[row][col] == ' ':
                return True
    return False

def hay_jugadas_disponibles(player):
    for row in range(board_size):
        for col in range(board_size):
            if es_casilla_valida(row, col, player):
                return True
    return False

def contar_fichas():
    count_W = sum(row.count('W') for row in board)
    count_B = sum(row.count('B') for row in board)
    if count_W > count_B:
        print("¡El jugador blanco (W) gana!")
    elif count_B > count_W:
        print("¡El jugador negro (B) gana!")
    else:
        print("¡Empate!")


def es_casilla_valida(row, col, player):
    if board[row][col] != ' ':
        return False
    directions = [(0, 1), (1, 0), (0, -1), (-1, 0), (1, 1), (-1, -1), (1, -1), (-1, 1)]

    for dr, dc in directions:
        y, x = row + dr, col + dc
        if not (0 <= y < board_size) or not (0 <= x < board_size):
            continue

        if board[y][x] != get_opponent(player):
            continue

        while 0 <= y < board_size and 0 <= x < board_size and board[y][x] == get_opponent(player):
            y += dr
            x += dc

        if 0 <= y < board_size and 0 <= x < board_size and board[y][x] == player:
            return True

    return False

def get_opponent(player):
    return 'W' if player == 'B' else 'B'


def crear_casilla(tablero, row, col, color=default_color):
    button = tk.Button(tablero, text='', command=lambda r=row, c=col: pintar_casilla(r, c), width=3, bg=color)

    if board[row][col] == 'W':
        button.config(bg=white_color, state='disabled')
    elif board[row][col] == 'B':
        button.config(bg=black_color, state='disabled')
    elif es_casilla_valida(row, col, current_color):
        button.config(bg='green')

    button.grid(row=row + 1, column=col + 1)
    return button


def cambiar_color():
    global current_color
    current_color = 'W' if current_color == 'B' else 'B'
    label_color.config(text=f'Color actual: {current_color}')

def imprimir_coordenadas(row, col):
    print(f"Posición seleccionada: ({row}, {col})")
def pintar_casilla(row, col):
    if board[row][col] == ' ' and es_casilla_valida(row, col, current_color):
        color = white_color if current_color == 'W' else black_color
        direction_check, check = check_mov_direction(row, col, color)
        if check:
            coors_repaints = coor_to_paint(row, col, direction_check, color)
            for coor_repaint in coors_repaints:
                for y, x in coor_repaint:
                    if board[y][x] != current_color:
                        board[y][x] = current_color
                        crear_casilla(tablero, y, x, color)
            imprimir_coordenadas(row, col)
            cambiar_color()
            darw_map()
            finalizar_juego()


    
def darw_map():
    for widget in tablero.winfo_children():
        widget.destroy()

    for i in range(board_size):
        tk.Label(tablero, text=str(i + 1), width=3).grid(row=i + 1, column=0)
        tk.Label(tablero, text=chr(ord('A') + i), width=3).grid(row=0, column=i + 1)

    for row in range(board_size):
        for col in range(board_size):
            if (row == 3 and col == 3) or (row == 4 and col == 4):
                crear_casilla(tablero, row, col, 'white')
            elif (row == 3 and col == 4) or (row == 4 and col == 3):
                crear_casilla(tablero, row, col, 'black')
            elif es_casilla_valida(row, col, current_color):
                crear_casilla(tablero, row, col, 'green')
            else:
                crear_casilla(tablero, row, col)


root = tk.Tk()
root.title('Othello')

tablero = tk.Frame(root)
tablero.pack()

current_color = 'B'

label_color = tk.Label(root, text=f'Color actual: {current_color}')
label_color.pack()


darw_map()
root.mainloop()


Direcciones disponibles ['E']
La dirección E filas y col 3 2
Las coordenadas a repintar son [(3, 2), (3, 3)]
Coordenadas para pintar [[(3, 2), (3, 3)]]
Posición seleccionada: (3, 2)
Direcciones disponibles ['S', 'SO']
La dirección S filas y col 2 4
Las coordenadas a repintar son [(2, 4), (3, 4)]
Coordenadas para pintar [[(2, 4), (3, 4)]]
La dirección SO filas y col 2 4
Las coordenadas a repintar son [(2, 4), (3, 3)]
Coordenadas para pintar [[(2, 4), (3, 4)], [(2, 4), (3, 3)]]
Posición seleccionada: (2, 4)
Direcciones disponibles ['E', 'SE', 'S']
La dirección E filas y col 2 3
Las coordenadas a repintar son [(2, 3), (2, 4)]
Coordenadas para pintar [[(2, 3), (2, 4)]]
La dirección SE filas y col 2 3
Las coordenadas a repintar son [(2, 3), (3, 4)]
Coordenadas para pintar [[(2, 3), (2, 4)], [(2, 3), (3, 4)]]
La dirección S filas y col 2 3
Las coordenadas a repintar son [(2, 3), (3, 3)]
Coordenadas para pintar [[(2, 3), (2, 4)], [(2, 3), (3, 4)], [(2, 3), (3, 3)]]
Posición seleccionada: (2, 

In [116]:
points_direction[1]

'NE'

In [117]:
board[2][3]

' '

In [118]:
from sys import maxsize

def min_max(state, player):
    if(player == "black"):
        v = []
        v.append(-maxsize)
        for action in actions(state):
            v.append(max_value(result(state, action)))
        return max(v)
    elif (player == "white"):
        v = []
        v.append(maxsize)
        for action in actions(state):
            v.append(min_value(result(state, action)))
        return min(v)        
    

def min_value(state):
    if(terminalState(state)):
        return utility(state)
    
    v = maxsize
    for action in actions(state):
        succesor =  result(state, action)
        eval_max = max_value(succesor)
        if(eval_max < v):
            v = eval_max
    return v

def max_value(state):
    if(terminalState(state)):
        return utility(state)
    
    v = -maxsize
    for action in actions(state):
        succesor =  result(state, action)
        eval_min = min_value(succesor)
        if(eval_max > v):
            v = eval_min
    return v

### CON PODA

In [119]:

def min_value(state, alpha, beta):
    if(terminalState(state)):
        return utility(state)
    v = -maxsize
    for action in actions(state):
        if(alpha >= beta):continue
        succesor =  result(state, action)
        eval_max = max_value(succesor, alpha, beta)
        if(eval_max < v):
            v = eval_max
            if((beta > v)):
                beta = v
    return v



def max_value(state, alpha, beta):
    if(terminalState(state)):
        return utility(state)
    v = -maxsize
    for action in actions(state):
        if(alpha >= beta):continue
        succesor =  result(state, action)
        eval_min = min_value(succesor, alpha, beta)
        if(eval_min > v):
            v = eval_min
            if((v > alpha)):
                alpha = v
    return v